In [ ]:
USE COVID

In [ ]:
SELECT TOP 10 * FROM sp_parquet.covid_recovered ORDER BY DATE DESC

In [ ]:
TRUNCATE TABLE dp.Covid_Development
INSERT INTO dp.Covid_Development
SELECT C.Country, c.[DATE] DTE, CONVERT(NVARCHAR(10), c.DATE, 103) DTE_Str, SUM(Confirmed) Confirmed,
 SUM(ISNULL(Recovered, 0)) Recovered, SUM(ISNULL(deaths, 0)) Deaths, SUM(Confirmed)-SUM(ISNULL(deaths, 0))-SUM(ISNULL(Recovered, 0)) Active
FROM sp_parquet.covid_confirmed c
     LEFT JOIN sp_parquet.covid_recovered r ON C.DATE=r.DATE AND c.country=r.country AND c.province=r.province
     LEFT JOIN sp_parquet.covid_deaths d ON C.DATE=d.DATE AND c.country=d.country AND c.province=d.province
GROUP BY C.Country, c.[DATE]

In [ ]:
SELECT count(*) FROM [dp].[Covid_Development]

In [ ]:
EXEC ('USE [Covid]; SELECT count(*)  FROM dp.Covid_Development') AT Data_Source SqlDataPool

In [ ]:
SELECT Top 10 * FROM dp.Covid_Development WHERE country = 'Germany' order by dte desc

In [ ]:
SELECT TOP 10 * FROM sp_csv.population

In [ ]:
SELECT DISTINCT Country
FROM dp.Covid_Development
WHERE NOT country IN(SELECT country FROM sp_csv.population)

In [ ]:
SELECT TOP 10 * FROM mi.Mapping_Country

In [ ]:
SELECT TOP 3 *
FROM mi.Mapping_Country
WHERE Ctry_Covid<>Ctry_Population
ORDER BY ctry_covid

In [ ]:
SELECT *
FROM(SELECT Dte, a.country, population, confirmed, active, active * 1000000 / population Active_PerMillion
     FROM dp.Covid_Development a
          INNER JOIN mi.Mapping_Country b ON a.country=b.ctry_covid
          INNER JOIN sp_csv.population c ON c.country=b.Ctry_Population
     WHERE dte IN (SELECT MAX(Date)FROM sp_parquet.covid_confirmed) and active > 0) a
ORDER BY Active_PerMillion DESC